# Creating Haiku requests 🪷

Our goal is to create a dataset which consists of the following:

- A prompt from a user requesting a haiku. We want these prompts to include simple examples like "I want a haiku about a cat" and more abstract like "Write a haiku about the impermanence of life".
- A set of haiku responses to this prompt.
- A ranking of the haiku responses.

In this notebook we'll focus on the first part of this process i.e. attempting to create diverse prompts systematically.

## Installation

For all of these notebooks we'll be using the excellent [distilabel](https://github.com/argilla-io/distilabel) library from Argilla which is described as an:
> AI Feedback (AIF) framework for building datasets with and for LLMs.

We'll explore more what this means in practice as we go through this notebook. Alongside this we'll also install the vLLM extra. [vLLM](https://github.com/vllm-project/vllm) is a library focused on efficient inference of LLMs. This is the library we'll use for running the models we use to create our data in this notebook. We can install both of these libraries by installing `distilabel` with the `vllm` extra.

In [1]:
%pip install distilabel['vllm'] -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
from datasets import Dataset
from distilabel.llm import vLLM
from distilabel.pipeline import Pipeline
from dotenv import load_dotenv
from huggingface_hub import hf_hub_download
from vllm import LLM
import os
import re

# Login to Hugging Face

We need to authenticate with the Hugging Face Hub if we want to push our datasets to the hub.

In [22]:
from huggingface_hub import login

login()

## Seed prompts for haiku

We'll start by creating a set of seed prompts for our haiku dataset. These will then be used to generate the actual prompts which aim to mirror the kinds of prompts a user might ask for. To generate the prompts from these seed terms we'll adapt a method from the [Self-Instruct: Aligning Language Models with Self-Generated Instructions](https://arxiv.org/abs/2212.10560). We'll get back to what that looks like in a moment.

For now, we'll generate a set of topics for haiku. For this I heavily leaned on Chat GPT but I also added a bunch myself. This could have been done with an open llm.

In [3]:
haiku_topics = [
    "mountain peaks",
    "moss",
    "fog over a pine forest",
    "the roaring sound of waves",
    "autumn leaves",
    "spring blossoms",
    "winter's first snow",
    "summer solstice",
    "cherry blossoms",
    "a quiet pond",
    "dew on grass",
    "a falling leaf",
    "the silent moon",
    "a tranquil river",
    "morning mist",
    "a butterfly's flight",
    "a spider's web",
    "desert sands",
    "a mountain stream",
    "forest at dawn",
    "a blooming rose",
    "a full moon",
    "a setting sun",
    "stars in the night sky",
    "a calm sea",
    "a gentle breeze",
    "a thunderstorm",
    "a snowy field",
    "a rainy day",
    "a rainbow",
    "a sunflower field",
    "a quiet beach",
    "a starry night",
    "a hummingbird",
    "a dragonfly",
    "a coral reef",
    "a bamboo grove",
    "a bird's song",
    "a winter's night",
    "a summer's day",
    "a spring morning",
    "an autumn sunset",
    "a lotus pond",
    "a pine forest",
    "a wildflower meadow",
    "a peaceful garden",
    "a koi pond",
    "a harvest moon",
    "a snowy mountain",
    "a rolling hill",
    "a gentle stream",
    "a quiet forest",
    "a blooming orchid",
    "a cactus flower",
    "a lily pad",
    "a firefly's glow",
    "a pebble's skip",
    "a seashell",
    "a tide pool",
    "a waterfall",
    "a cliff by the sea",
    "a desert oasis",
    "a field of lavender",
    "a glacier",
    "a hot spring",
    "a lightning bolt",
    "a meteor shower",
    "a misty valley",
    "a monsoon",
    "a northern light",
    "a rainbow eucalyptus",
    "a redwood forest",
    "a sand dune",
    "a tidal wave",
    "a volcano",
    "a willow tree",
    "a winter solstice",
    "a zephyr",
    "an iceberg",
    "an oak tree",
    "an orchard in bloom",
    "an underwater cave",
    "aurora borealis",
    "bioluminescent waves",
    "crystal formations",
    "falling petals",
    "geysers erupting",
    "hailstones",
    "ice crystals",
    "jungle canopy",
    "migrating birds",
    "morning dew",
    "mountain fog",
    "night blooming flowers",
    "ocean waves",
    "polar ice caps",
    "rain on a lake",
    "reef corals",
    "rippling water",
    "sakura petals",
    "snowflakes falling",
    "spring rain",
    "summer heatwave",
    "sunrise over mountains",
    "sunset on the beach",
    "the Milky Way",
    "the moon's reflection",
    "the sound of crickets",
    "thunder in the distance",
    "tropical rainforest",
    "wild geese flying",
    "winter frost",
    "a barn owl",
    "a beaver dam",
    "a bee pollinating",
    "a bird's nest",
    "a black bear",
    "a blue whale",
    "a brook trout",
    "a brown hare",
    "a butterfly cocoon",
    "a cactus wren",
    "a cat napping",
    "a chameleon",
    "a cheetah running",
    "a chipmunk",
    "a coral snake",
    "a coyote howl",
    "a crab scuttling",
    "a crane in flight",
    "a deer in the woods",
    "a dolphin leaping",
    "a dragonfly hovering",
    "a duckling",
    "a eagle soaring",
    "a earthworm",
    "a egret",
    "a elephant herd",
    "a elk bugling",
    "a falcon dive",
    "a fawn",
    "a fire ant",
    "a flamingo",
    "a fox in snow",
    "a frog croaking",
    "a gecko",
    "a giraffe",
    "a goldfish",
    "a gorilla",
    "a grasshopper",
    "a great blue heron",
    "a grizzly bear",
    "a hawk circling",
    "a hedgehog",
    "a hermit crab",
    "a honeybee",
    "a horse galloping",
    "a hummingbird feeding",
    "a hyena",
    "a iguana",
    "a jaguar",
    "a jellyfish",
    "a kangaroo",
    "a kingfisher",
    "a koala",
    "a komodo dragon",
    "a ladybug",
    "a lamb",
    "a lemur",
    "a leopard",
    "a lion roaring",
    "a lizard basking",
    "a lobster",
    "a lynx",
    "a macaw",
    "a manatee",
    "a meerkat",
    "a monarch butterfly",
    "a mongoose",
    "a moose",
    "a mosquito",
    "a mountain goat",
    "a mouse",
    "a narwhal",
    "a nightingale",
    "a octopus",
    "a orca",
    "a ostrich",
    "a otter",
    "a owl hooting",
    "a panda",
    "a panther",
    "a parakeet",
    "a parrot",
    "a peacock",
    "a pelican",
    "a penguin",
    "a peregrine falcon",
    "a pigeon",
    "a platypus",
    "a polar bear",
    "a porcupine",
    "a prairie dog",
    "a praying mantis",
    "a puffin",
    "a python",
    "a quail",
    "a rabbit",
    "a raccoon",
    "a rattlesnake",
    "a red fox",
    "a reindeer",
    "a rhinoceros",
    "a roadrunner",
    "a robin",
    "a salamander",
    "a scorpion",
    "a sea lion",
    "a sea turtle",
    "a seagull",
    "a seal",
    "a shark",
    "a sheep",
    "a skunk",
    "a sloth",
    "a snail",
    "a snake",
    "a snow leopard",
    "a snow owl",
    "a sparrow",
    "a spider",
    "a squirrel",
    "a starfish",
    "a stingray",
    "a stork",
    "a swan",
    "a tarantula",
    "a tiger",
    "a toad",
    "a toucan",
    "a tree frog",
    "a turkey",
    "a turtle",
    "a vulture",
    "a walrus",
    "a warthog",
    "a wasp",
    "a water buffalo",
    "a weasel",
    "a whale shark",
    "a wild boar",
    "a wolf howling",
    "a woodpecker",
    "a zebra",
    "an albatross",
    "an alligator",
    "an ant",
    "an antelope",
    "an armadillo",
    "an axolotl",
    "an eagle",
    "an echidna",
    "an elephant seal",
    "an emu",
    "an ibex",
    "an iguana",
    "an impala",
    "an octopus",
    "an opossum",
    "an orangutan",
    "an ostrich",
    "an otter",
    "an owl",
    "an ox",
    "an oyster",
    "an pangolin",
    "an parrotfish",
    "an penguin",
    "an platypus",
    "an porpoise",
    "an quetzal",
    "an rabbit",
    "an raccoon",
    "an rat",
    "an reindeer",
    "an rhino",
    "an seahorse",
    "an seal",
    "an shark",
    "an sloth",
    "an snail",
    "an squid",
    "an squirrel",
    "an starfish",
    "an stingray",
    "an tapir",
    "an tarantula",
    "an tiger",
    "an toucan",
    "an turtle",
    "an urial",
    "an vulture",
    "an walrus",
    "an warthog",
    "an whale",
    "an wolf",
    "an yak",
    "an zebra",
    "bamboo in the wind",
    "beach at sunset",
    "birds chirping at dawn",
    "blooming cherry trees",
    "blossoms in the breeze",
    "blue sky with white clouds",
    "butterflies in a meadow",
    "calm lake at dawn",
    "canyon echoes",
    "cherry blossoms at night",
    "clouds over mountains",
    "cold mountain stream",
    "crickets at night",
    "dandelion seeds blowing",
    "dawn chorus of birds",
    "desert under the stars",
    "dew-covered spiderweb",
    "distant thunder",
    "early spring flowers",
    "evening rain",
    "fallen leaves in a stream",
    "fireflies at dusk",
    "first frost",
    "first spring rain",
    "flock of migrating birds",
    "flowering cactus",
    "foggy morning",
    "forest in autumn",
    "frost on a window",
    "frozen lake",
    "full moon over water",
    "gentle rain on leaves",
    "geese flying south",
    "glacier melting",
    "golden autumn leaves",
    "harvest moon night",
    "hazy summer day",
    "high tide",
    "hiking in the mountains",
    "ice on a branch",
    "icy river",
    "insects buzzing",
    "lake at night",
    "late autumn chill",
    "leaves in the wind",
    "lightning over the ocean",
    "lilacs in bloom",
    "lonely mountain path",
    "long summer days",
    "loons on a lake",
    "maple leaves in fall",
    "meadow in bloom",
    "midnight sky",
    "mist over the hills",
    "moonlit beach",
    "morning sun on dew",
    "mountain in the clouds",
    "night sky with stars",
    "northern lights",
    "ocean at night",
    "ocean breeze",
    "old forest",
    "orange autumn sunset",
    "orchids blooming",
    "pine trees in snow",
    "pond with lily pads",
    "quiet winter night",
    "rain on a tin roof",
    "rainbow after rain",
    "red leaves of autumn",
    "river in spring",
    "rocks in a stream",
    "roses in bloom",
    "rustling leaves",
    "sakura in full bloom",
    "sand dunes",
    "sea at dawn",
    "seashore at low tide",
    "season's first snowfall",
    "silent snowfall",
    "snow-covered hills",
    "snowy owl in flight",
    "soft autumn rain",
    "spring brook",
    "spring cherry blossoms",
    "springtime garden",
    "starlit night",
    "stormy sea",
    "summer night sky",
    "sun breaking through clouds",
    "sunrise over the ocean",
    "sunset over the lake",
    "swaying bamboo",
    "thunder and lightning",
    "tranquil forest stream",
    "tropical beach",
    "tulips in spring",
    "waves crashing on rocks",
    "whale breaching",
    "wildflowers in spring",
    "wind in the pines",
    "winter moon",
    "wisteria in bloom",
    "woodland in autumn",
    "zen garden",
]


abstract_topics = [
    "the melancholy at the end of summer",
    "the serenity of rain",
    "whispers of the autumn wind",
    "the stillness of a frozen lake",
    "the first bloom of spring",
    "shadows dancing in moonlight",
    "the quiet of a snowfall",
    "dawn's first light",
    "the hush of a foggy morning",
    "twilight's last gleaming",
    "the infinity of the starry sky",
    "the gentle touch of a breeze",
    "ripples on a still pond",
    "the passage of time in the mountains",
    "the solitude of a desert",
    "the mystery of the northern lights",
    "the warmth of the morning sun",
    "the dance of fireflies at dusk",
    "the whisper of leaves in fall",
    "the rebirth of the forest in spring",
    "the echo of a distant thunder",
    "the calm before the storm",
    "the beauty of a butterfly's flight",
    "the silent language of flowers",
    "the fleeting beauty of a rainbow",
    "the embrace of night's darkness",
    "the purity of a snowflake's design",
    "the transformation of caterpillar to butterfly",
    "the reflection of mountains in a lake",
    "the secrets held in ancient rocks",
    "the rhythm of the ocean waves",
    "the journey of a drifting cloud",
    "the grace of a swan on water",
    "the bond between the moon and the tide",
    "the allure of a path less traveled",
    "the mystery of the fading twilight",
    "the harmony of birds at dawn",
    "the solitude of a lighthouse",
    "the patience of a blooming flower",
    "the rush of a cascading waterfall",
    "the peace of a sleeping village",
    "the joy of leaves in the wind",
    "the passage of seasons",
    "the contrast of lightning in a storm",
    "the cycle of life and death in nature",
    "the whisper of a gentle stream",
    "the warmth of a campfire in the cold",
    "the unity of a flock of birds",
    "the first snow of winter",
    "the resilience of life in the wild",
    "the glow of the sunset on the ocean",
    "the mystery of the deep forest",
    "the beauty of the night sky",
    "the silence of a world asleep",
    "the first light of dawn on a mountain",
    "the simplicity of a single leaf",
    "the endurance of a rock",
    "the fleeting moment of a shooting star",
    "the ancient wisdom of trees",
    "the delicate balance of nature",
    "the quiet power of the moon",
    "the joy of a sunbeam breaking through clouds",
    "the sorrow of a wilting flower",
    "the hope in a new bud",
    "the majesty of a thunderstorm",
    "the serenity of a garden",
    "the journey of a river to the sea",
    "the dance of sunlight on water",
    "the whisper of the night wind",
    "the mystery of a fog-covered landscape",
    "the beauty of a snow-covered field",
    "the transformation of day to night",
    "the quietude of a forest at dusk",
    "the first chirp before dawn",
    "the solitude of a moonlit night",
    "the gentle closing of a day",
    "the promise of a sunrise",
    "the reflection of the sky in a lake",
    "the story told by ancient ruins",
    "the dance of leaves in a storm",
    "the embrace of the earth and sky",
    "the passage of a comet",
    "the whisper of an old forest",
    "the serenity of a mountain peak",
    "the fleeting touch of a raindrop",
    "the echo of a canyon",
    "the warmth of the sun after rain",
    "the beauty in a petal's curve",
    "the rhythm of falling leaves",
    "the mystery of twilight shadows",
    "the calm of a starry night",
    "the silence in the eye of a hurricane",
    "the first breath of spring",
    "the nostalgia evoked by a falling leaf",
    "the eternity in a grain of sand",
    "the dance of waves against the shore",
    "the whisper of a winter's night",
    "the embrace of the earth's rhythm",
    "the fleeting nature of a dream",
]


cs_haiku_topics = [
    "computer science",
    "machine learning",
    "deep learning",
    "artificial intelligence",
    "computer vision",
    "natural language processing",
    "data science",
    "Hugging Face",
    "PyTorch",
    "TensorFlow",
    "Keras",
    "NumPy",
    "Pandas",
    "Matplotlib",
    "Scikit-learn",
    "SciPy",
    "PySpark",
    "Apache Spark",
    "Apache Hadoop",
    "Apache Kafka",
    "Apache Airflow",
    "Apache Arrow",
    "Apache Beam",
    "Apache Cassandra",
    "Apache CouchDB",
    "dask",
    "DVC",
    "fast.ai",
    "fastText",
    "gensim",
    "JAX",
    "Kubeflow",
    "NLTK",
    "OpenCV",
    "OpenNLP",
    "OpenAI",
    "OpenNMT",
]

haiku_phrases = [
    "Morning dew on grass",
    "Whisper of autumn leaves",
    "Gentle river flow",
    "Moonlit silent night",
    "Sunset's fiery glow",
    "Snowflakes' delicate dance",
    "Blossoms in spring breeze",
    "Mountain's stoic stance",
    "Ocean's rhythmic waves",
    "Butterflies' gentle flight",
    "Rain's soft serenade",
    "Stars twinkling at night",
    "Sunrise over hills",
    "Frost's art on window panes",
    "Quiet forest trails",
    "Summer's first warm rain",
    "Crisp air of fall morn",
    "Birdsong at dawn's light",
    "Winter's first snowfall",
    "Full moon's brilliant sight",
    "Cherry blossoms bloom",
    "Rustling of fallen leaves",
    "Distant thunder's boom",
    "Calm before the storm",
    "Icy river's flow",
    "Fields of golden corn",
    "Misty mountain air",
    "Autumn's harvest moon",
    "Spring's fresh floral flair",
    "Lonely winter path",
    "Sunset on the beach",
    "Fog's mysterious wrath",
    "Rainbow after rain",
    "Quiet snowy evening",
    "Desert's vast domain",
    "Glistening morning dew",
    "Bare trees in winter's grip",
    "Summer's deep sky blue",
    "Harvest moon's soft light",
    "Geese flying in V-form",
    "Stars' dance through the night",
    "Waves kissing the shore",
    "Breeze through autumn leaves",
    "Thunderstorm's fierce roar",
    "Pine scent in cool air",
    "Fireflies' twilight dance",
    "Old barn, standing bare",
    "Creek's babbling song",
    "Leaves crunching underfoot",
    "Days growing shorter, long",
    "Hawk soaring above",
    "Frost-kissed morning fields",
    "Nature's acts of love",
    "Lone flower in bloom",
    "Icicles' slow drip",
    "Night's enveloping gloom",
    "Sun's caress on skin",
    "Meadow's green expanse",
    "New day to begin",
    "Petals on the breeze",
    "Lightning splits the sky",
    "Autumn's brisk, cool tease",
    "Silent snowy town",
    "Bare branches against sky",
    "Leaves turning brown",
    "Dew on spider's web",
    "Moon's reflection in lake",
    "Harvest's ebb and flow",
    "Fog over the fields",
    "Lighthouse in the storm",
    "The peace that nature yields",
    "Wildflowers in bloom",
    "Crisp scent of pine trees",
    "Evening's quiet gloom",
    "Ripples on a pond",
    "Glow of the setting sun",
    "Nature's bond",
    "Hush of the forest",
    "Beach's endless horizon",
    "Winter's cold caress",
    "Path through the meadow",
    "Raindrops on the window",
    "Gentle deer in shadow",
    "Frost's early morning",
    "Stars fading at dawn",
    "Day's new beginning",
    "Chirping of crickets",
    "Clouds drifting lazily",
    "Nature's tickets",
    "Winds whispering tales",
    "Misty hills at dawn",
    "Sailboat's distant sails",
    "Night's velvet curtain",
    "Snow's blanket, soft and white",
    "Spring's certain return",
]


To make sure we don't have duplicates, we shove all of the topics into a set. We could also do some other similarity measures to remove very similar prompts but for now we'll keep things simple.

We'll now create a Hugging Face `datasets.Dataset` from this list. The distilabel library heavily uses the 🤗 `datasets` library under the hood. This would also mean if we wanted to work with data already on the Hub we could easily use the `load_dataset` function to grab the data, and as we'll see later, we can also easily upload our data to the Hub easily using the `push_to_hub` method.

In [4]:

all_topics = list(set(haiku_topics + abstract_topics + cs_haiku_topics + haiku_phrases))

dataset = Dataset.from_dict(
    {
        "input": haiku_topics + abstract_topics + cs_haiku_topics + haiku_phrases,
    }
)


## Using a modified Self Instruct to generate prompts

The Self Instruct paper introduces a method that we can use to help generate prompts via an LLM without having to write them all by hand. Self Instruct aims to generate prompts from an application description. Let's write one for a haiku generation model. 

In [1]:
from distilabel.tasks import SelfInstructTask

In [2]:
application_description = (
    "An AI assistant adept at writing Haiku. "
    "It expects complete suggestions from users providing details of the kind of haiku they want. "
    "The AI assistant will help users write haiku about particular topics and is willing to accept requests related to a specific subject or object or a more abstract request"
    "based on an emotion, theme or vibe."
)

Let's take a look at the default criteria for query generation from the self instruct paper:

In [3]:
print(SelfInstructTask.criteria_for_query_generation)

Incorporate a diverse range of verbs, avoiding repetition.
Ensure queries are compatible with AI model's text generation functions and are limited to 1-2 sentences.
Design queries to be self-contained and standalone.
Blend interrogative (e.g., "What is the significance of x?") and imperative (e.g., "Detail the process of x.") styles.


MOst of this is okay but the last line doesn't really make sense for our use case. We can instead adapt this to be something like:

In [13]:
criteria_queries = (
    "Incorporate a diverse range of verbs, avoiding repetition.\n"
    "Ensure queries are compatible with AI model's text generation functions and are limited to 1-2 sentences.\n"
    "Design queries to be self-contained and standalone."
)

We can now create our task passing in our criteria and application description. This task is used by `distilabel` to define what we're trying to achieve and helps us avoid having to write a lot of boilerplate code ourselves.

In [16]:
instruction_task = SelfInstructTask(
    system_prompt="You are an expert Haiku writer, writing the best and most diverse Haiku given topics as inputs.",
    application_description=application_description,
    criteria_for_query_generation=criteria_queries,
    num_instructions=15,
)

## Creating our LLM

Another really nice feature of the `distilabel` library is that we can use different LLMs for generation. This includes closed models like OpenAI models, open models we can run through Hugging Face's inference API and models we can run locally. In this case we'll use `vLLM` to run the models locally. I started by using [llama.cpp](https://github.com/ggerganov/llama.cpp) (via the Python client library) and running all the code locally on a Macbook. This worked fine for development but for the final larger run the GPU version of vLLM was much faster.

We can construct a generator i.e. the actual LLM by using the distilabel `vLLM` Class to wrap the vLLM model. We also pass in the task, the prompt_format and some other variables which are passed to the model.

### Model choices

Currently there are, over 46,000 models with the text-generation task on the Hub. Whilst we can spend a lot of time evaluating these models side by side as part of our process, to keep things more focused we'll only use on model for this stage of the work. Here are some possible heuristics and steps that can help choose a model.

I had a few criteria for choosing a model:
- I only wanted to use open models.
- For this project I wanted a model that is fairly light i.e. something that I could potentially run on a local laptop (MacBook Pro M1) without too much trouble. This tends to mean models <=7B parameters.
- I wanted to use a model with an architecture that is well supported by different inference libraries and services. This doesn't exclude much but may remove more experimental or research focused models.

Based on these criteria the following can be helpful tools for deciding on a model:

- [Hugging Chat](https://huggingface.co/chat/) provides a really effort free way to try out a bunch of models. Most of the models hosted here are models with a lot of community interest and are likely to be good starting points for consideration.
- [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard) is a leaderboard of open models that have been evaluated on a number of tasks. You can filter this by model size and a range of other filters
- [r/LocalLLaMA/](https://www.reddit.com/r/LocalLLaMA/) is a subreddit dedicated to local inference of LLMs. This is a great place to ask questions about running models locally and to see what other people are doing. You can often get a sense from following discussions here which models work well for particular tasks.

For this project I decided to use the [OpenHermes-2.5-Mistral-7B](https://huggingface.co/teknium/OpenHermes-2.5-Mistral-7B) model. This is a 7B parameter model that I have used a lot for other projects. There may be better 7B models out there already but to start with I wanted to not spend to much time on this part of the project.

Since we want to make our generation process as GPU efficient we'll use an `AWQ` quantized version of the model. This will reduce the memory requirements of the model further allowing us to run the model on a smaller GPU if needed.

In [18]:
llm = vLLM(
    model=LLM(model="TheBloke/OpenHermes-2.5-Mistral-7B-AWQ"),
    task=instruction_task,
    max_new_tokens=128,
    temperature=0.4,
    prompt_format="chatml",
)

WARNING 02-29 13:22:37 config.py:186] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 02-29 13:22:37 llm_engine.py:79] Initializing an LLM engine with config: model='TheBloke/OpenHermes-2.5-Mistral-7B-AWQ', tokenizer='TheBloke/OpenHermes-2.5-Mistral-7B-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 02-29 13:22:37 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 02-29 13:22:42 llm_engine.py:337] # GPU blocks: 14185, # CPU blocks: 2048
INFO 02-29 13:22:42 model_runner.py:676] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-29 13:22:42 model_runner.py:680] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 02-29 13:22:55 model_runner.py:748] Graph capturing finished in 13 secs.


We now create a `Pipeline`, this can potentially be used to run multiple models in sequence but for now we'll just use it to run our generator.

In [19]:
pipeline = Pipeline(generator=llm)

Once we have the `Pipeline` created we can run our actual generation. We pass in the `seed_prompts` we created earlier and the number of prompts we want to generate.

In [20]:
distiset = pipeline.generate(
    dataset=dataset,
    num_generations=15,
    shuffle_before_labelling=False,
    batch_size=4,
    display_progress_bar=True,
)


INFO:distilabel:Executing dry-run...
INFO:distilabel:Processing batch 1 of 1...
INFO:distilabel:Calling generator for batch 1...


Flattening the indices:   0%|          | 0/1 [00:00<?, ? examples/s]

INFO:distilabel:Dry-run executed with no issues. Starting the actual generation...


Output()

INFO:distilabel:Processing batch 1 of 161...
INFO:distilabel:Calling generator for batch 1...
INFO:distilabel:Processing batch 2 of 161...
INFO:distilabel:Calling generator for batch 2...
INFO:distilabel:Processing batch 3 of 161...
INFO:distilabel:Calling generator for batch 3...
INFO:distilabel:Processing batch 4 of 161...
INFO:distilabel:Calling generator for batch 4...
INFO:distilabel:Processing batch 5 of 161...
INFO:distilabel:Calling generator for batch 5...
INFO:distilabel:Processing batch 6 of 161...
INFO:distilabel:Calling generator for batch 6...
INFO:distilabel:Processing batch 7 of 161...
INFO:distilabel:Calling generator for batch 7...
INFO:distilabel:Processing batch 8 of 161...
INFO:distilabel:Calling generator for batch 8...
INFO:distilabel:Processing batch 9 of 161...
INFO:distilabel:Calling generator for batch 9...
INFO:distilabel:Processing batch 10 of 161...
INFO:distilabel:Calling generator for batch 10...
INFO:distilabel:Processing batch 11 of 161...
INFO:distila

Flattening the indices:   0%|          | 0/644 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/644 [00:00<?, ? examples/s]

INFO:distilabel:Final dataset saved at /content/ckpt


As you can see below we get back a `Dataset` with the prompts we generated. We can now save this to disk and upload it to the Hub. We'll use the config variable of the `push_to_hub` method to put these prompts in a particular config. This is a super nice way of keeping our datasets organised and not having to create a new repo for each dataset we create.

In [21]:
distiset

Dataset({
    features: ['input', 'generation_model', 'generation_prompt', 'raw_generation_responses', 'instructions'],
    num_rows: 644
})

In [ ]:
# distiset.push_to_hub("davanstrien/haiku_dpo", "raw_prompts", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/9.03k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/davanstrien/haiku_dpo/commit/98931b178814fda3a947dafb72f14bd3ce4b3694', commit_message='Upload dataset', commit_description='', oid='98931b178814fda3a947dafb72f14bd3ce4b3694', pr_url=None, pr_revision=None, pr_num=None)

## Cleaning up

We'll now to a little bit of cleaning to make sure the prompts are in the right format.

In [23]:
def transform(inst: str) -> str:
    """Remove 1., 2., ... from the instruction."""
    clean_inst = re.sub(r"^\d+\.\s*", "", inst)
    return f"{clean_inst}"

In [24]:
instructions = []
for generations in distiset["raw_generation_responses"]:
    instructions.extend(
        transform(prompt) for prompt in generations[0].split("\n") if prompt != ""
    )

We'll now also create a new dataset that just contains the prompts. This will be used in the next notebook to generate the actual haiku.

In [25]:
prompt_dataset = Dataset.from_dict({"instructions": instructions})

In [26]:
prompt_dataset

Dataset({
    features: ['instructions'],
    num_rows: 4748
})

In [27]:
# prompt_dataset.push_to_hub("davanstrien/haiku_prompts", private=True)